# Step 7: LSTM Deep Learning Model Training (PyTorch)

## Goal
Train a Long Short-Term Memory (LSTM) neural network using PyTorch to predict BIST-100 price direction, leveraging temporal patterns that traditional ML models might miss.

## Why LSTM?
- **Temporal Dependencies**: LSTM networks excel at capturing long-term dependencies in time series data
- **Sequence Learning**: Can learn complex patterns across multiple time steps
- **Non-linear Relationships**: Deep learning can capture non-linear relationships between features
- **Lagged Macro Features**: Incorporates 1-month and 3-month lagged inflation and interest rates

## Why PyTorch?
- **Python 3.14 Compatible**: Works with the latest Python versions
- **Flexible**: Easy to customize model architecture
- **Performance**: Efficient training with GPU support (if available)

## Models
- **LSTM Neural Network (PyTorch)**: Multi-layer LSTM with dropout and batch normalization
- **Comparison**: Compare with XGBoost baseline

## Evaluation Metrics
- Accuracy, Precision, Recall, F1-Score
- Confusion Matrix
- Training History Visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import subprocess
import warnings
warnings.filterwarnings('ignore')

# Check Python version
python_version = sys.version_info
print(f"🐍 Python version: {python_version.major}.{python_version.minor}.{python_version.micro}")

# PyTorch for LSTM training (supports Python 3.14)
PYTORCH_AVAILABLE = False
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader
    PYTORCH_AVAILABLE = True
    print(f"✅ PyTorch {torch.__version__} available")
    if torch.cuda.is_available():
        print(f"   🚀 CUDA available - GPU acceleration enabled")
    else:
        print(f"   💻 Using CPU")
except ImportError:
    print("📦 PyTorch not found. Attempting to install...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "--quiet"])
        import torch
        import torch.nn as nn
        import torch.optim as optim
        from torch.utils.data import Dataset, DataLoader
        PYTORCH_AVAILABLE = True
        print(f"✅ PyTorch {torch.__version__} installed successfully")
        if torch.cuda.is_available():
            print(f"   🚀 CUDA available - GPU acceleration enabled")
        else:
            print(f"   💻 Using CPU")
    except Exception as e:
        print(f"❌ PyTorch installation failed: {e}")
        print("   Please install manually: python -m pip install torch")
        raise ImportError("PyTorch is required for LSTM training")

# Machine Learning (for comparison)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
import joblib

# XGBoost (for comparison)
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("⚠️  XGBoost not installed. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Paths
current_dir = Path().resolve()
if current_dir.name == "notebooks":
    project_root = current_dir.parent
else:
    project_root = current_dir

data_processed_dir = project_root / "data" / "processed"
models_dir = project_root / "models"
models_dir.mkdir(parents=True, exist_ok=True)
reports_dir = project_root / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)

print("✅ LSTM training setup complete!")
print(f"   Project root: {project_root}")
print(f"   Processed data dir: {data_processed_dir}")
print(f"   Models dir: {models_dir}")
print(f"   Framework: PyTorch")

🐍 Python version: 3.14.2
✅ PyTorch 2.9.1+cpu available
   💻 Using CPU
✅ LSTM training setup complete!
   Project root: C:\Users\cihan\turkish_finance_ml
   Processed data dir: C:\Users\cihan\turkish_finance_ml\data\processed
   Models dir: C:\Users\cihan\turkish_finance_ml\models
   Framework: PyTorch


## 1. Load and Prepare Data

In [ ]:
# Load processed data
print("📊 Loading processed data...")

# Load full feature dataset
full_features_file = data_processed_dir / "bist_features_full.csv"
if not full_features_file.exists():
    raise FileNotFoundError(f"File not found: {full_features_file}. Run preprocessing notebook first.")

df = pd.read_csv(full_features_file)
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

print(f"✅ Loaded full dataset: {df.shape}")

# Load macro features if available
macro_file = data_processed_dir / "bist_macro_merged.csv"
if macro_file.exists():
    print(f"\n📊 Loading macroeconomic features...")
    macro_df = pd.read_csv(macro_file)
    macro_df['Date'] = pd.to_datetime(macro_df['Date'])
    
    # Merge macro features
    macro_cols = [col for col in macro_df.columns if col not in ['Date', 'BIST100_Close']]
    df = df.merge(macro_df[['Date'] + macro_cols], on='Date', how='left')
    df[macro_cols] = df[macro_cols].ffill().bfill()
    print(f"   ✅ Added {len(macro_cols)} macro features: {', '.join(macro_cols)}")

# Select features (exclude Date, Ticker, and targets)
exclude_cols = ['Date']
if 'Ticker' in df.columns:
    exclude_cols.append('Ticker')

target_cols = [col for col in df.columns if col.startswith('Target_')]
feature_cols = [col for col in df.columns if col not in exclude_cols + target_cols]

# Create feature matrix and target
X = df[feature_cols].copy()
y = df['Target_Direction'].copy()

# Remove rows with NaN
valid_idx = ~(X.isnull().any(axis=1) | y.isnull())
X = X[valid_idx].reset_index(drop=True)
y = y[valid_idx].reset_index(drop=True)

print(f"\n📊 Dataset Summary:")
print(f"   Total samples: {len(X)}")
print(f"   Features: {X.shape[1]}")
print(f"   Target distribution:")
print(f"      Up (1): {np.sum(y == 1):,} ({np.sum(y == 1)/len(y)*100:.2f}%)")
print(f"      Down (0): {np.sum(y == 0):,} ({np.sum(y == 0)/len(y)*100:.2f}%)")

# Scale features
print(f"\n🔄 Scaling features...")
scaler = MinMaxScaler()  # Use MinMaxScaler for neural networks
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns,
    index=X.index
)
print(f"   ✅ Features scaled to [0, 1] range")

📊 Loading processed data...
✅ Loaded full dataset: (6015, 77)

📊 Dataset Summary:
   Total samples: 6015
   Features: 70
   Target distribution:
      Up (1): 2,475 (41.15%)
      Down (0): 3,540 (58.85%)

🔄 Scaling features...
   ✅ Features scaled to [0, 1] range


## 2. Create Sequences for LSTM

LSTM requires sequences of data. We'll create sequences where each sample contains N previous time steps.

In [ ]:
def create_sequences(X, y, sequence_length=30):
    """
    Create sequences for LSTM training
    
    Args:
        X: Feature matrix (DataFrame or array)
        y: Target vector
        sequence_length: Number of time steps to look back
    
    Returns:
        X_seq: 3D array (samples, timesteps, features)
        y_seq: Target array
    """
    X_seq = []
    y_seq = []
    
    for i in range(sequence_length, len(X)):
        X_seq.append(X.iloc[i-sequence_length:i].values)
        y_seq.append(y.iloc[i])
    
    return np.array(X_seq), np.array(y_seq)

# Create sequences
SEQUENCE_LENGTH = 30  # Look back 30 days
print(f"📅 Creating sequences with {SEQUENCE_LENGTH} time steps...")

X_seq, y_seq = create_sequences(X_scaled, y, sequence_length=SEQUENCE_LENGTH)

print(f"✅ Sequences created!")
print(f"   X_seq shape: {X_seq.shape} (samples, timesteps, features)")
print(f"   y_seq shape: {y_seq.shape}")

# Time series split (80/20)
split_idx = int(len(X_seq) * 0.8)
X_train_seq = X_seq[:split_idx]
X_test_seq = X_seq[split_idx:]
y_train_seq = y_seq[:split_idx]
y_test_seq = y_seq[split_idx:]

print(f"\n📊 Train/Test Split:")
print(f"   Training: {len(X_train_seq):,} sequences")
print(f"   Testing: {len(X_test_seq):,} sequences")
print(f"   Features per timestep: {X_train_seq.shape[2]}")

📅 Creating sequences with 30 time steps...
✅ Sequences created!
   X_seq shape: (5985, 30, 70) (samples, timesteps, features)
   y_seq shape: (5985,)

📊 Train/Test Split:
   Training: 4,788 sequences
   Testing: 1,197 sequences
   Features per timestep: 70


## 3. Build and Train LSTM Model

In [ ]:
if PYTORCH_AVAILABLE:
    print("🧠 Building LSTM model (PyTorch)...")
    print("="*60)
    
    # Model architecture
    n_features = X_train_seq.shape[2]
    
    # Define PyTorch LSTM model
        class LSTMModel(nn.Module):
            def __init__(self, input_size, hidden_size1=128, hidden_size2=64, hidden_size3=32, num_layers=1, dropout=0.2):
                super(LSTMModel, self).__init__()
                self.lstm1 = nn.LSTM(input_size, hidden_size1, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
                self.dropout1 = nn.Dropout(dropout)
                self.lstm2 = nn.LSTM(hidden_size1, hidden_size2, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
                self.dropout2 = nn.Dropout(dropout)
                self.lstm3 = nn.LSTM(hidden_size2, hidden_size3, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
                self.dropout3 = nn.Dropout(dropout)
                self.bn = nn.BatchNorm1d(hidden_size3)
                self.fc1 = nn.Linear(hidden_size3, 32)
                self.dropout_fc = nn.Dropout(dropout)
                self.fc2 = nn.Linear(32, 16)
                self.fc3 = nn.Linear(16, 1)
                self.sigmoid = nn.Sigmoid()
                
            def forward(self, x):
                # First LSTM layer
                lstm_out1, _ = self.lstm1(x)
                lstm_out1 = self.dropout1(lstm_out1)
                
                # Second LSTM layer
                lstm_out2, _ = self.lstm2(lstm_out1)
                lstm_out2 = self.dropout2(lstm_out2)
                
                # Third LSTM layer - take last timestep
                lstm_out3, _ = self.lstm3(lstm_out2)
                lstm_out3 = lstm_out3[:, -1, :]  # Take last timestep
                lstm_out3 = self.dropout3(lstm_out3)
                lstm_out3 = self.bn(lstm_out3)
                
                # Dense layers
                out = torch.relu(self.fc1(lstm_out3))
                out = self.dropout_fc(out)
                out = torch.relu(self.fc2(out))
                out = self.sigmoid(self.fc3(out))
                return out
        
        # Create model
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = LSTMModel(n_features).to(device)
        print(f"✅ Model architecture (PyTorch) - Device: {device}")
        print(f"   Total parameters: {sum(p.numel() for p in model.parameters()):,}")
        
        # Convert data to PyTorch tensors
        X_train_tensor = torch.FloatTensor(X_train_seq).to(device)
        y_train_tensor = torch.FloatTensor(y_train_seq.reshape(-1, 1)).to(device)
        X_test_tensor = torch.FloatTensor(X_test_seq).to(device)
        y_test_tensor = torch.FloatTensor(y_test_seq.reshape(-1, 1)).to(device)
        
        # Create validation split
        val_size = int(len(X_train_tensor) * 0.2)
        X_val_tensor = X_train_tensor[-val_size:]
        y_val_tensor = y_train_tensor[-val_size:]
        X_train_tensor = X_train_tensor[:-val_size]
        y_train_tensor = y_train_tensor[:-val_size]
        
        # Training setup
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=0.00001)
        
        # Training loop
        print(f"\n🚀 Training LSTM model (PyTorch)...")
        print("="*60)
        
        epochs = 50
        batch_size = 32
        best_val_loss = float('inf')
        patience = 10
        patience_counter = 0
        history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
        
        for epoch in range(epochs):
            # Training
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0
            
            for i in range(0, len(X_train_tensor), batch_size):
                batch_X = X_train_tensor[i:i+batch_size]
                batch_y = y_train_tensor[i:i+batch_size]
                
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                predicted = (outputs > 0.5).float()
                train_total += batch_y.size(0)
                train_correct += (predicted == batch_y).sum().item()
            
            # Validation
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for i in range(0, len(X_val_tensor), batch_size):
                    batch_X = X_val_tensor[i:i+batch_size]
                    batch_y = y_val_tensor[i:i+batch_size]
                    
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y)
                    
                    val_loss += loss.item()
                    predicted = (outputs > 0.5).float()
                    val_total += batch_y.size(0)
                    val_correct += (predicted == batch_y).sum().item()
            
            train_loss /= (len(X_train_tensor) // batch_size + 1)
            val_loss /= (len(X_val_tensor) // batch_size + 1)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total
            
            history['loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['accuracy'].append(train_acc)
            history['val_accuracy'].append(val_acc)
            
            scheduler.step(val_loss)
            
            if (epoch + 1) % 5 == 0 or epoch == 0:
                print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_state = model.state_dict().copy()
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    model.load_state_dict(best_model_state)
                    break
        
        model.load_state_dict(best_model_state)
        print("✅ Training complete!")
        
else:
    print("❌ PyTorch not available. Cannot train LSTM model.")
    print("   Please install PyTorch: python -m pip install torch")
    model = None
    history = None

IndentationError: unexpected indent (711338683.py, line 9)

## 4. Evaluate LSTM Model

In [ ]:
if PYTORCH_AVAILABLE and model is not None:
    # PyTorch predictions
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train_tensor).cpu().numpy()
        test_pred = model(X_test_tensor).cpu().numpy()
        y_train_pred_lstm = (train_pred > 0.5).astype(int).flatten()
        y_test_pred_lstm = (test_pred > 0.5).astype(int).flatten()
        y_test_proba_lstm = test_pred.flatten()
    
    # Calculate metrics
    train_accuracy_lstm = accuracy_score(y_train_seq, y_train_pred_lstm)
    test_accuracy_lstm = accuracy_score(y_test_seq, y_test_pred_lstm)
    test_precision_lstm = precision_score(y_test_seq, y_test_pred_lstm)
    test_recall_lstm = recall_score(y_test_seq, y_test_pred_lstm)
    test_f1_lstm = f1_score(y_test_seq, y_test_pred_lstm)
    test_auc_lstm = roc_auc_score(y_test_seq, y_test_proba_lstm)
    
    print("="*60)
    print("📊 LSTM MODEL PERFORMANCE (PyTorch)")
    print("="*60)
    print(f"\n   Training Accuracy: {train_accuracy_lstm:.4f} ({train_accuracy_lstm*100:.2f}%)")
    print(f"   Test Accuracy: {test_accuracy_lstm:.4f} ({test_accuracy_lstm*100:.2f}%)")
    print(f"   Test Precision: {test_precision_lstm:.4f}")
    print(f"   Test Recall: {test_recall_lstm:.4f}")
    print(f"   Test F1-Score: {test_f1_lstm:.4f}")
    print(f"   Test AUC-ROC: {test_auc_lstm:.4f}")
    
    # Confusion Matrix
    cm_lstm = confusion_matrix(y_test_seq, y_test_pred_lstm)
    
    plt.figure(figsize=(12, 5))
    
    # Confusion Matrix
    plt.subplot(1, 2, 1)
    sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('LSTM Confusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Training History
    plt.subplot(1, 2, 2)
    plt.plot(history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history['val_accuracy'], label='Val Accuracy', linewidth=2)
    plt.plot(history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history['val_loss'], label='Val Loss', linewidth=2)
    plt.title('LSTM Training History', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(reports_dir / 'lstm_performance.png', dpi=300, bbox_inches='tight')
    print(f"\n💾 Saved visualization to: {reports_dir / 'lstm_performance.png'}")
    plt.show()
else:
    print("❌ LSTM model not available for evaluation.")

❌ LSTM model not available for evaluation.


## 5. Train XGBoost Baseline for Comparison

In [ ]:
# Prepare data for XGBoost (use last timestep of each sequence)
print("🌲 Training XGBoost baseline for comparison...")
print("="*60)

# Use the last timestep of each sequence as features for XGBoost
X_train_xgb = X_train_seq[:, -1, :]  # Last timestep
X_test_xgb = X_test_seq[:, -1, :]

if XGBOOST_AVAILABLE:
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='logloss',
        scale_pos_weight=(np.sum(y_train_seq == 0) / np.sum(y_train_seq == 1))
    )
    
    xgb_model.fit(X_train_xgb, y_train_seq)
    
    # Predictions
    y_train_pred_xgb = xgb_model.predict(X_train_xgb)
    y_test_pred_xgb = xgb_model.predict(X_test_xgb)
    y_test_proba_xgb = xgb_model.predict_proba(X_test_xgb)[:, 1]
    
    # Calculate metrics
    train_accuracy_xgb = accuracy_score(y_train_seq, y_train_pred_xgb)
    test_accuracy_xgb = accuracy_score(y_test_seq, y_test_pred_xgb)
    test_precision_xgb = precision_score(y_test_seq, y_test_pred_xgb)
    test_recall_xgb = recall_score(y_test_seq, y_test_pred_xgb)
    test_f1_xgb = f1_score(y_test_seq, y_test_pred_xgb)
    test_auc_xgb = roc_auc_score(y_test_seq, y_test_proba_xgb)
    
    print("✅ XGBoost training complete!")
    print(f"\n   Training Accuracy: {train_accuracy_xgb:.4f} ({train_accuracy_xgb*100:.2f}%)")
    print(f"   Test Accuracy: {test_accuracy_xgb:.4f} ({test_accuracy_xgb*100:.2f}%)")
    print(f"   Test Precision: {test_precision_xgb:.4f}")
    print(f"   Test Recall: {test_recall_xgb:.4f}")
    print(f"   Test F1-Score: {test_f1_xgb:.4f}")
    print(f"   Test AUC-ROC: {test_auc_xgb:.4f}")
else:
    print("⚠️  XGBoost not available. Skipping comparison.")
    xgb_model = None

🌲 Training XGBoost baseline for comparison...
✅ XGBoost training complete!

   Training Accuracy: 0.9576 (95.76%)
   Test Accuracy: 0.4854 (48.54%)
   Test Precision: 0.4778
   Test Recall: 0.8875
   Test F1-Score: 0.6212
   Test AUC-ROC: 0.5179


## 6. Model Comparison: LSTM vs XGBoost

In [ ]:
if PYTORCH_AVAILABLE and XGBOOST_AVAILABLE and model is not None and xgb_model is not None:
    print("="*60)
    print("📊 MODEL COMPARISON: LSTM vs XGBoost")
    print("="*60)
    
    comparison_data = {
        'Model': ['LSTM', 'XGBoost'],
        'Test Accuracy': [test_accuracy_lstm, test_accuracy_xgb],
        'Test Precision': [test_precision_lstm, test_precision_xgb],
        'Test Recall': [test_recall_lstm, test_recall_xgb],
        'Test F1-Score': [test_f1_lstm, test_f1_xgb],
        'Test AUC-ROC': [test_auc_lstm, test_auc_xgb]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    display(comparison_df)
    
    # Determine best model
    if test_accuracy_lstm > test_accuracy_xgb:
        best_model_name = "LSTM"
        improvement = (test_accuracy_lstm - test_accuracy_xgb) * 100
        print(f"\n🏆 Best Model: LSTM")
        print(f"   Improvement over XGBoost: +{improvement:.2f} percentage points")
    else:
        best_model_name = "XGBoost"
        improvement = (test_accuracy_xgb - test_accuracy_lstm) * 100
        print(f"\n🏆 Best Model: XGBoost")
        print(f"   Improvement over LSTM: +{improvement:.2f} percentage points")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Metrics comparison
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
    lstm_scores = [test_accuracy_lstm, test_precision_lstm, test_recall_lstm, test_f1_lstm, test_auc_lstm]
    xgb_scores = [test_accuracy_xgb, test_precision_xgb, test_recall_xgb, test_f1_xgb, test_auc_xgb]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    axes[0].bar(x - width/2, lstm_scores, width, label='LSTM', alpha=0.8)
    axes[0].bar(x + width/2, xgb_scores, width, label='XGBoost', alpha=0.8)
    axes[0].set_xlabel('Metrics')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(metrics, rotation=45, ha='right')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    axes[0].set_ylim([0, 1])
    
    # ROC Curves
    fpr_lstm, tpr_lstm, _ = roc_curve(y_test_seq, y_test_proba_lstm)
    fpr_xgb, tpr_xgb, _ = roc_curve(y_test_seq, y_test_proba_xgb)
    
    axes[1].plot(fpr_lstm, tpr_lstm, label=f'LSTM (AUC = {test_auc_lstm:.3f})', linewidth=2)
    axes[1].plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {test_auc_xgb:.3f})', linewidth=2)
    axes[1].plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title('ROC Curves Comparison', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(reports_dir / 'lstm_vs_xgboost.png', dpi=300, bbox_inches='tight')
    print(f"\n💾 Saved comparison visualization to: {reports_dir / 'lstm_vs_xgboost.png'}")
    plt.show()
    
    print("\n💡 Insights:")
    if test_accuracy_lstm > test_accuracy_xgb:
        print("   ✅ LSTM captures temporal patterns better than XGBoost")
        print("   ✅ Deep learning approach shows promise for time series prediction")
    else:
        print("   ⚠️  XGBoost performs better with current feature engineering")
        print("   💡 Consider: Longer sequences, more LSTM layers, or feature engineering")
    
else:
    print("⚠️  Cannot compare: Missing LSTM or XGBoost model")

⚠️  Cannot compare: Missing LSTM or XGBoost model


## 7. Save Models

In [ ]:
print("💾 Saving models...")

if PYTORCH_AVAILABLE and model is not None:
    # Save PyTorch model
    lstm_model_path = models_dir / "lstm_model.pth"
    torch.save(model.state_dict(), lstm_model_path)
    print(f"   ✅ LSTM model (PyTorch) saved: {lstm_model_path}")
    
    # Save model architecture info for loading later
    model_info = {
        'input_size': n_features,
        'hidden_size1': 128,
        'hidden_size2': 64,
        'hidden_size3': 32,
        'dropout': 0.2,
        'sequence_length': SEQUENCE_LENGTH
    }
    import json
    with open(models_dir / "lstm_model_info.json", 'w') as f:
        json.dump(model_info, f, indent=2)
    print(f"   ✅ Model info saved: {models_dir / 'lstm_model_info.json'}")
    
    # Save scaler
    scaler_path = models_dir / "lstm_scaler.pkl"
    joblib.dump(scaler, scaler_path)
    print(f"   ✅ Scaler saved: {scaler_path}")

if XGBOOST_AVAILABLE and xgb_model is not None:
    # Save XGBoost model
    xgb_model_path = models_dir / "xgb_baseline_lstm.pkl"
    joblib.dump(xgb_model, xgb_model_path)
    print(f"   ✅ XGBoost baseline saved: {xgb_model_path}")

print("\n✅ All models saved!")

💾 Saving models...
   ✅ XGBoost baseline saved: C:\Users\cihan\turkish_finance_ml\models\xgb_baseline_lstm.pkl

✅ All models saved!


## 8. Summary

### Key Findings:
- **LSTM Architecture (PyTorch)**: Multi-layer LSTM (128→64→32 units) with dropout and batch normalization
- **Sequence Length**: 30 days lookback period
- **Features**: 70+ technical indicators + lagged macroeconomic features (Inflation, Interest Rates with 1M and 3M lags)
- **Performance**: Compared with XGBoost baseline

### Advantages of LSTM:
1. **Temporal Memory**: Can remember patterns across long sequences (30 days)
2. **Non-linear Patterns**: Deep learning captures complex relationships
3. **Feature Learning**: Automatically learns relevant features from sequences
4. **Lagged Features**: Incorporates delayed economic impacts (1-month and 3-month lags)

### Framework:
- **PyTorch**: Compatible with Python 3.14, flexible architecture, GPU support

### Next Steps:
- Experiment with different sequence lengths (15, 30, 60 days)
- Try bidirectional LSTM for better pattern recognition
- Add attention mechanisms for important time steps
- Ensemble LSTM with XGBoost for improved accuracy
- Fine-tune hyperparameters (learning rate, dropout, hidden sizes)